In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
gc.collect()
import datetime
import warnings
warnings.filterwarnings("ignore")

## Loading Data Using Kaggle Api 

In [ ]:
! pip install -q kaggle

from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ayushpattiwar","key":"0528b921a4e9d9c2da41dd72cdfef52d"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c elo-merchant-category-recommendation

100% 545M/548M [00:11<00:00, 77.3MB/s]
100% 548M/548M [00:11<00:00, 49.2MB/s]
  0% 0.00/17.2k [00:00<?, ?B/s]
100% 17.2k/17.2k [00:00<00:00, 17.2MB/s]
  0% 0.00/3.02M [00:00<?, ?B/s]
100% 3.02M/3.02M [00:00<00:00, 97.5MB/s]
  0% 0.00/846k [00:00<?, ?B/s]
100% 846k/846k [00:00<00:00, 114MB/s]
 91% 45.0M/49.4M [00:01<00:00, 20.7MB/s]
100% 49.4M/49.4M [00:01<00:00, 40.8MB/s]
  0% 0.00/1.13M [00:00<?, ?B/s]
100% 1.13M/1.13M [00:00<00:00, 157MB/s]
 71% 9.00M/12.7M [00:01<00:00, 7.22MB/s]
100% 12.7M/12.7M [00:01<00:00, 10.9MB/s]
  0% 0.00/17.2k [00:00<?, ?B/s]
100% 17.2k/17.2k [00:00<00:00, 18.9MB/s]


In [ ]:
# unzipping the dataset
! unzip '/content/historical_transactions.csv.zip'

! unzip '/content/merchants.csv.zip'

! unzip '/content/new_merchant_transactions.csv.zip'

! unzip '/content/train.csv.zip'

! unzip '/content/test.csv.zip'

Archive:  /content/historical_transactions.csv.zip
  inflating: historical_transactions.csv  
Archive:  /content/merchants.csv.zip
  inflating: merchants.csv           
Archive:  /content/new_merchant_transactions.csv.zip
  inflating: new_merchant_transactions.csv  
Archive:  /content/train.csv.zip
  inflating: train.csv               
Archive:  /content/test.csv.zip
  inflating: test.csv                


In [ ]:
np.random.seed(123)
gc.collect()

15

In [ ]:
# https://www.kaggle.com/c/champs-scalar-coupling/discussion/96655
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
# loading train and test using reduce_mem_usage and Pandas
train = reduce_mem_usage(pd.read_csv('/content/train.csv'))
test = reduce_mem_usage(pd.read_csv('/content/test.csv'))

Memory usage after optimization is: 4.04 MB
Decreased by 56.2%
Memory usage after optimization is: 2.24 MB
Decreased by 52.5%


In [ ]:
# checking for null values
train.isnull().sum()

first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64

In [ ]:
# imputing missing values with mode
# https://stackoverflow.com/questions/42789324/pandas-fillna-mode
test['first_active_month'].fillna(test['first_active_month'].mode()[0], inplace=True)

In [ ]:
# checking for null values
test.isnull().sum()

first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64

In [ ]:
# converting date features to datetime
train['first_active_month']=pd.to_datetime(train['first_active_month'])
test['first_active_month']=pd.to_datetime(test['first_active_month'])

In [ ]:
# making a new columns with outliers as seen in eda notebook
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1

# https://www.geeksforgeeks.org/mean-encoding-machine-learning/
# mean encoding categorical features by grouping them with outliers.
for feature in ['feature_1', 'feature_2', 'feature_3']:
  mapping = train.groupby([feature])['outliers'].mean()
  train[feature] = train[feature].map(mapping)
  test[feature] = test[feature].map(mapping)

In [ ]:
# https://www.kaggle.com/mks2192/feature-engineering
train['quarter']=train['first_active_month'].dt.quarter
train['total_time'] = (datetime.datetime.today() - train['first_active_month']).dt.days
train['start_month'] = train['first_active_month'].dt.month
train['start_year'] = train['first_active_month'].dt.year
train['dayofweek'] = train['first_active_month'].dt.dayofweek
train['quarter']=train['first_active_month'].dt.quarter

train['total_time_feature1']=train['total_time']*train['feature_1']
train['total_time_feature2']=train['total_time']*train['feature_2']
train['total_time_feature3']=train['total_time']*train['feature_3']

train['total_time_feature1_ratio']=(train['feature_1']/train['total_time'])
train['total_time_feature2_ratio']=(train['feature_2']/train['total_time'])
train['total_time_feature3_ratio']=(train['feature_3']/train['total_time'])

# getting aggregated features from categorical variables
train['feature_sum'] = train['feature_1'] + train['feature_2'] + train['feature_3']
train['feature_mean'] = train['feature_sum']/3
train['feature_max'] = train[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
train['feature_min'] = train[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
train['feature_var'] = train[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

gc.collect()

61

In [ ]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,quarter,total_time,start_month,start_year,dayofweek,total_time_feature1,total_time_feature2,total_time_feature3,total_time_feature1_ratio,total_time_feature2_ratio,total_time_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820312,0,2,1325,6,2017,3,17.416615,11.596561,15.141465,0.000010,0.000007,0.000009,0.033324,0.011108,0.013145,0.008752,0.002214
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392822,0,1,1476,1,2017,6,15.810309,16.803926,15.177464,0.000007,0.000008,0.000007,0.032379,0.010793,0.011385,0.010283,0.000555
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.687988,0,3,1629,8,2016,0,17.283510,14.257205,16.750738,0.000007,0.000005,0.000006,0.029645,0.009882,0.010610,0.008752,0.000992
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142456,0,3,1233,9,2017,4,13.207393,17.467174,12.678736,0.000009,0.000011,0.000008,0.035161,0.011720,0.014166,0.010283,0.002129
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159790,0,4,1172,11,2017,2,9.444546,16.603024,12.051483,0.000007,0.000012,0.000009,0.032508,0.010836,0.014166,0.008058,0.003091


In [ ]:
# https://www.kaggle.com/mks2192/feature-engineering
test['quarter']=test['first_active_month'].dt.quarter
test['total_time'] = (datetime.datetime.today() - test['first_active_month']).dt.days
test['start_month'] = test['first_active_month'].dt.month
test['start_year'] = test['first_active_month'].dt.year
test['dayofweek'] = test['first_active_month'].dt.dayofweek
test['quarter']=test['first_active_month'].dt.quarter

test['total_time_feature1']=test['total_time']*test['feature_1']
test['total_time_feature2']=test['total_time']*test['feature_2']
test['total_time_feature3']=test['total_time']*test['feature_3']

test['total_time_feature1_ratio']=(test['feature_1']/test['total_time'])
test['total_time_feature2_ratio']=(test['feature_2']/test['total_time'])
test['total_time_feature3_ratio']=(test['feature_3']/test['total_time'])

# getting aggregated features from categorical variables
test['feature_sum'] = test['feature_1'] + test['feature_2'] + test['feature_3']
test['feature_mean'] = test['feature_sum']/3
test['feature_max'] = test[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
test['feature_min'] = test[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
test['feature_var'] = test[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

gc.collect()

22

In [ ]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter,total_time,start_month,start_year,dayofweek,total_time_feature1,total_time_feature2,total_time_feature3,total_time_feature1_ratio,total_time_feature2_ratio,total_time_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var
0,2017-04-01,C_ID_0ab67a22ab,0.010479,0.014166,0.011428,2,1386,4,2017,5,14.524432,19.634634,15.838544,0.000008,0.000010,0.000008,0.036073,0.012024,0.014166,0.010479,0.001915
1,2017-01-01,C_ID_130fd0cbdd,0.010610,0.014166,0.010283,1,1476,1,2017,6,15.660197,20.909610,15.177464,0.000007,0.000010,0.000007,0.035059,0.011686,0.014166,0.010283,0.002154
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011385,0.011428,3,1264,8,2017,1,16.614794,14.390354,14.444386,0.000010,0.000009,0.000009,0.035957,0.011986,0.013145,0.011385,0.001004
3,2017-12-01,C_ID_d27d835a9f,0.010610,0.011385,0.010283,4,1142,12,2017,4,12.116494,13.001412,11.742998,0.000009,0.000010,0.000009,0.032277,0.010759,0.011385,0.010283,0.000566
4,2015-12-01,C_ID_2b5e3df5c2,0.013145,0.011385,0.011428,4,1873,12,2015,1,24.619865,21.323682,21.403746,0.000007,0.000006,0.000006,0.035957,0.011986,0.013145,0.011385,0.001004


In [ ]:
train.shape

(201917, 23)

In [ ]:
test.shape

(123623, 21)

## Merchants

In [ ]:
# loading csv file
merchant=reduce_mem_usage(pd.read_csv('/content/merchants.csv'))

Memory usage after optimization is: 30.32 MB
Decreased by 46.0%


In [ ]:
# dropping columns which dont gives much information based on eda
merchant.drop(columns=['merchant_category_id','subsector_id', 'city_id', 'state_id'], axis = 1, inplace=True)
merchant.drop(columns=['numerical_1'], axis = 1, inplace=True)

In [ ]:
# replacing inf values with nan
merchant = merchant.replace([np.inf, -np.inf], np.nan)

In [ ]:
# checking for null values
merchant.isnull().sum()

merchant_id                        0
merchant_group_id                  0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 3
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 3
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                3
active_months_lag12                0
category_4                         0
category_2                     11887
dtype: int64

In [ ]:
# imputing missing values with mode
merchant['avg_sales_lag3'].fillna(merchant['avg_sales_lag3'].mode()[0], inplace=True)
merchant['avg_sales_lag6'].fillna(merchant['avg_sales_lag6'].mode()[0], inplace=True)
merchant['avg_sales_lag12'].fillna(merchant['avg_sales_lag12'].mode()[0], inplace=True)
merchant['category_2'].fillna(merchant['category_2'].mode()[0], inplace=True)

In [ ]:
# preprocessing merchant file
merchant['category_1'] = merchant['category_1'].map({'Y':1, 'N':0})
merchant['category_4'] = merchant['category_4'].map({'Y':1, 'N':0})
merchant['most_recent_sales_range'] = merchant['most_recent_sales_range'].map({'E':4, 'D':3, 'C':2, 'B':1, 'A':0})
merchant['most_recent_purchases_range'] = merchant['most_recent_purchases_range'].map({'E':4, 'D':3, 'C':2, 'B':1, 'A':0})

In [ ]:
merchant.head()

,merchant_id,merchant_group_id,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_2
0,M_ID_838061e48c,8353,-0.057465,0,4,4,-0.400000,9.666667,3,-2.250000,18.666667,6,-2.320000,13.916667,12,0,1.0
1,M_ID_9339d880ad,3184,-0.057465,0,4,4,-0.720000,1.750000,3,-0.740000,1.291667,6,-0.570000,1.687500,12,0,1.0
2,M_ID_e726bbae1e,447,-0.057465,0,4,4,-82.129997,260.000000,2,-82.129997,260.000000,2,-82.129997,260.000000,2,0,5.0
3,M_ID_a70e9c5f81,5026,-0.057465,1,4,4,1.000000,1.666667,3,1.000000,4.666667,6,1.000000,3.833333,12,1,1.0
4,M_ID_64456c37ce,2228,-0.057465,1,4,4,1.000000,0.500000,3,1.000000,0.361111,6,1.000000,0.347222,12,1,1.0


## Historical_transactions

In [ ]:
# loading the data
hist_trans = reduce_mem_usage(pd.read_csv('/content/historical_transactions.csv'))

# preprocessing the csv file
# imputing the missing values
hist_trans['category_3'].fillna(hist_trans['category_3'].mode()[0], inplace=True)
hist_trans['merchant_id'].fillna(hist_trans['merchant_id'].mode()[0], inplace=True)
hist_trans['category_2'].fillna(hist_trans['category_2'].mode()[0], inplace=True)

# mapping catrgorical variables
hist_trans['authorized_flag'] = hist_trans['authorized_flag'].map({'Y':1, 'N':0})
hist_trans['category_1'] = hist_trans['category_1'].map({'Y':1, 'N':0})
hist_trans['category_3'] = hist_trans['category_3'].map({'A':0, 'B':1, 'C':2})
hist_trans['installments'] = hist_trans['installments'].map({-1:13, 0:0.1,1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:10,11:11,12:12,999:13})

# taking 99 percrntile value a max to remove the outliers
hist_trans['purchase_amount'] = hist_trans['purchase_amount'].apply(lambda x: min(x, 1.22))

# feature engineering based on dates
hist_trans['purchase_date']=pd.to_datetime(hist_trans['purchase_date'])
hist_trans['year'] = hist_trans['purchase_date'].dt.year
hist_trans['day'] = hist_trans['purchase_date'].dt.day
hist_trans['month'] = hist_trans['purchase_date'].dt.month
hist_trans['dayofweek'] = hist_trans['purchase_date'].dt.dayofweek
hist_trans['weekofyear'] = hist_trans['purchase_date'].dt.weekofyear
hist_trans['hour_of_purchase'] = hist_trans['purchase_date'].dt.hour
hist_trans['Minute_of_purchase'] = hist_trans['purchase_date'].dt.minute
hist_trans['Second_of_purchase'] = hist_trans['purchase_date'].dt.second
hist_trans['purchased_on_weekend'] = (hist_trans.dayofweek >=5).astype(int)
hist_trans['purchased_on_weekday'] = (hist_trans.dayofweek <5).astype(int)

hist_trans['month_diff'] = ((datetime.datetime.today() - hist_trans['purchase_date']).dt.days)//30
hist_trans['month_diff'] += hist_trans['month_lag']

# feature engineering based on installments and purchase amount
# purchase amount is highly normalized so we denormalizing it  
# inspired from https://chandureddyvari.com/posts/elo-merchant-feature/
hist_trans['EMI'] = hist_trans['purchase_amount'] / hist_trans['installments']
hist_trans['purchase_amount_quantiles'] = pd.qcut(hist_trans['purchase_amount'], 5, labels=False)
hist_trans['duration'] = hist_trans['purchase_amount']*hist_trans['month_diff']
hist_trans['amount_month_ratio'] = hist_trans['purchase_amount']/hist_trans['month_diff']

hist_trans = reduce_mem_usage(hist_trans)

# aggregating by grouping them by card_id.
aggregations = {
    'purchase_date' : ['max','min'],
    'purchased_on_weekend': ['sum', 'mean'],
    'purchased_on_weekday': ['sum', 'mean'],
    'dayofweek' : ['nunique', 'sum', 'mean','max'],
    'hour_of_purchase': ['nunique', 'mean', 'min', 'max'],
    'Minute_of_purchase': ['nunique', 'mean', 'min', 'max'],
    'Second_of_purchase': ['nunique', 'mean', 'min', 'max'],
    'weekofyear': ['nunique', 'mean', 'min', 'max'],
    'month_diff': ['max','min','mean','var','skew'],
    'day': ['nunique', 'sum', 'min'],
    'month' : ['sum', 'mean', 'nunique','max'],
    'purchase_amount_quantiles' : ['var', 'mean', 'skew'],
    'duration' : ['mean','min','max','var','skew'],
    'amount_month_ratio' : ['mean','min','max','var','skew'],
    'authorized_flag' : ['sum','mean'],
    'subsector_id': ['nunique'],
    'card_id': ['size'],
    'city_id' : ['nunique'],
    'state_id' : ['nunique'],
    'merchant_id': ['nunique'],
    'installments': ['sum','max','mean','var','skew'],
    'merchant_category_id': ['nunique'],
    'purchase_amount': ['sum', 'mean', 'min', 'max', 'var','skew'],
    'EMI' : ['sum','mean','max','min','var'],
    'category_1' : ['sum','mean', 'max','min'],
    'category_2' : ['sum','mean'],
    'category_3' : ['sum','mean'],
    'month_lag' : ['sum','max','min','mean','var','skew']
}
aggregated_trans = hist_trans.groupby('card_id').agg(aggregations)
aggregated_trans.columns = ['transactions_'+'_'.join(col).strip()
                           for col in aggregated_trans.columns.values]
aggregated_trans.reset_index(inplace=True)

# extracting some more features based on aggregated features.
aggregated_trans['transactions_purchase_date_diff'] = (aggregated_trans['transactions_purchase_date_max']-aggregated_trans['transactions_purchase_date_min']).dt.days
aggregated_trans['transactions_purchase_date_average'] = aggregated_trans['transactions_purchase_date_diff']/aggregated_trans['transactions_card_id_size']
aggregated_trans['transactions_purchase_date_uptonow'] = (datetime.datetime.today()-aggregated_trans['transactions_purchase_date_max']).dt.days
aggregated_trans['transactions_purchase_date_uptomin'] = (datetime.datetime.today()-aggregated_trans['transactions_purchase_date_min']).dt.days

Memory usage after optimization is: 1749.11 MB
Decreased by 43.7%
Memory usage after optimization is: 1638.06 MB
Decreased by 69.4%


In [ ]:
aggregated_trans.head()

,card_id,transactions_purchase_date_max,transactions_purchase_date_min,transactions_purchased_on_weekend_sum,transactions_purchased_on_weekend_mean,transactions_purchased_on_weekday_sum,transactions_purchased_on_weekday_mean,transactions_dayofweek_nunique,transactions_dayofweek_sum,transactions_dayofweek_mean,transactions_dayofweek_max,transactions_hour_of_purchase_nunique,transactions_hour_of_purchase_mean,transactions_hour_of_purchase_min,transactions_hour_of_purchase_max,transactions_Minute_of_purchase_nunique,transactions_Minute_of_purchase_mean,transactions_Minute_of_purchase_min,transactions_Minute_of_purchase_max,transactions_Second_of_purchase_nunique,transactions_Second_of_purchase_mean,transactions_Second_of_purchase_min,transactions_Second_of_purchase_max,transactions_weekofyear_nunique,transactions_weekofyear_mean,transactions_weekofyear_min,transactions_weekofyear_max,transactions_month_diff_max,transactions_month_diff_min,transactions_month_diff_mean,transactions_month_diff_var,transactions_month_diff_skew,transactions_day_nunique,transactions_day_sum,transactions_day_min,transactions_month_sum,transactions_month_mean,transactions_month_nunique,transactions_month_max,transactions_purchase_amount_quantiles_var,...,transactions_subsector_id_nunique,transactions_card_id_size,transactions_city_id_nunique,transactions_state_id_nunique,transactions_merchant_id_nunique,transactions_installments_sum,transactions_installments_max,transactions_installments_mean,transactions_installments_var,transactions_installments_skew,transactions_merchant_category_id_nunique,transactions_purchase_amount_sum,transactions_purchase_amount_mean,transactions_purchase_amount_min,transactions_purchase_amount_max,transactions_purchase_amount_var,transactions_purchase_amount_skew,transactions_EMI_sum,transactions_EMI_mean,transactions_EMI_max,transactions_EMI_min,transactions_EMI_var,transactions_category_1_sum,transactions_category_1_mean,transactions_category_1_max,transactions_category_1_min,transactions_category_2_sum,transactions_category_2_mean,transactions_category_3_sum,transactions_category_3_mean,transactions_month_lag_sum,transactions_month_lag_max,transactions_month_lag_min,transactions_month_lag_mean,transactions_month_lag_var,transactions_month_lag_skew,transactions_purchase_date_diff,transactions_purchase_date_average,transactions_purchase_date_uptonow,transactions_purchase_date_uptomin
0,C_ID_00007093c1,2018-02-27 05:14:57,2017-02-14 14:00:43,25.0,0.167785,124.0,0.832215,7,334.0,2.241611,6,18,14.416107,0,22,57,30.308725,0,59,54,28.791946,0,59,39,25.550336,1,52,36,35,35.140940,0.121894,2.084855,28,1974.0,1,950.0,6.375839,12,12,1.393162,...,13,149,4,3,29,192.00000,6.0,1.289062,0.584961,3.310547,18,-77.1250,-0.517578,-0.729004,1.219727,0.081543,3.632812,-74.31250,-0.498779,0.406738,-0.729004,0.062866,28.0,0.187919,1,0,393.0,2.636719,173.0,1.161074,-872.0,0,-12,-5.852349,11.923998,0.135031,377,2.530201,1053,1431
1,C_ID_0001238066,2018-02-27 16:18:59,2017-09-28 22:25:14,52.0,0.422764,71.0,0.577236,7,431.0,3.504065,6,20,14.739837,0,23,47,28.390244,0,59,55,28.243902,0,59,23,29.967480,1,52,36,35,35.016260,0.016127,7.744375,30,1985.0,1,899.0,7.308943,6,12,1.740104,...,17,123,18,6,65,240.00000,13.0,1.951172,5.113281,3.601562,29,-72.4375,-0.588867,-0.734863,0.768066,0.036194,4.058594,-62.59375,-0.508789,0.128052,-0.734863,0.059479,2.0,0.016260,1,0,203.0,1.650391,152.0,1.235772,-223.0,0,-5,-1.813008,1.661469,-0.239327,151,1.227642,1052,1204
2,C_ID_0001506ef0,2018-02-17 12:33:56,2017-01-14 16:16:01,32.0,0.484848,34.0,0.515152,7,236.0,3.575758,6,15,12.606061,0,21,40,29.666667,0,59,36,29.969697,0,58,24,27.090909,1,51,36,35,35.121212,0.108159,2.375525,25,792.0,1,449.0,6.803030,11,12,1.822145,...,12,66,3,2,28,7.50000,1.0,0.113586,0.012276,8.125000,19,-34.8750,-0.528320,-0.740723,1.219727,0.207275,2.544922,-342.25000,-5.187500,12.203125,-7.406250,20.984375,0.0,0.000000,0,0,194.0,2.939453,1.0,0.015152,-319.0,0,-13,-4.833333,17.956410,-0.723706,398

## New_merchant_transctions

In [ ]:
# loading the data
new_hist_trans = reduce_mem_usage(pd.read_csv('/content/new_merchant_transactions.csv'))

# preprocessing the csv file
# imputing the missing values
new_hist_trans['category_3'].fillna(new_hist_trans['category_3'].mode()[0], inplace=True)
new_hist_trans['merchant_id'].fillna(new_hist_trans['merchant_id'].mode()[0], inplace=True)
new_hist_trans['category_2'].fillna(new_hist_trans['category_2'].mode()[0], inplace=True)

# mapping catrgorical variables
new_hist_trans['authorized_flag'] = new_hist_trans['authorized_flag'].map({'Y':1, 'N':0})
new_hist_trans['category_1'] = new_hist_trans['category_1'].map({'Y':1, 'N':0})
new_hist_trans['category_3'] = new_hist_trans['category_3'].map({'A':0, 'B':1, 'C':2})
new_hist_trans['installments'] = new_hist_trans['installments'].map({-1:13, 0:0.1,1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:10,11:11,12:12,999:13})

# taking 99 percrntile value a max to remove the outliers
new_hist_trans['purchase_amount'] = new_hist_trans['purchase_amount'].apply(lambda x: min(x, 1.22))

# feature engineering based on dates
new_hist_trans['purchase_date']=pd.to_datetime(new_hist_trans['purchase_date'])
new_hist_trans['year'] = new_hist_trans['purchase_date'].dt.year
new_hist_trans['day'] = new_hist_trans['purchase_date'].dt.day
new_hist_trans['month'] = new_hist_trans['purchase_date'].dt.month
new_hist_trans['dayofweek'] = new_hist_trans['purchase_date'].dt.dayofweek
new_hist_trans['weekofyear'] = new_hist_trans['purchase_date'].dt.weekofyear
new_hist_trans['hour_of_purchase'] = new_hist_trans['purchase_date'].dt.hour
new_hist_trans['Minute_of_purchase'] = new_hist_trans['purchase_date'].dt.minute
new_hist_trans['Second_of_purchase'] = new_hist_trans['purchase_date'].dt.second
new_hist_trans['purchased_on_weekend'] = (new_hist_trans.dayofweek >=5).astype(int)
new_hist_trans['purchased_on_weekday'] = (new_hist_trans.dayofweek <5).astype(int)

new_hist_trans['month_diff'] = ((datetime.datetime.today() - new_hist_trans['purchase_date']).dt.days)//30
new_hist_trans['month_diff'] += new_hist_trans['month_lag']

# feature engineering based on installments and purchase amount
# purchase amount is highly normalized so we denormalizing it  
# inspired from https://chandureddyvari.com/posts/elo-merchant-feature/
new_hist_trans['EMI'] = new_hist_trans['purchase_amount'] / new_hist_trans['installments']
new_hist_trans['purchase_amount_quantiles'] = pd.qcut(new_hist_trans['purchase_amount'], 5, labels=False)
new_hist_trans['duration'] = new_hist_trans['purchase_amount']*new_hist_trans['month_diff']
new_hist_trans['amount_month_ratio'] = new_hist_trans['purchase_amount']/new_hist_trans['month_diff']

new_hist_trans = reduce_mem_usage(new_hist_trans)

# aggregating by grouping them by card_id.
aggregations = {
    'purchase_date' : ['max','min'],
    'purchased_on_weekend': ['sum', 'mean'],
    'purchased_on_weekday': ['sum', 'mean'],
    'dayofweek' : ['nunique', 'sum', 'mean','max'],
    'hour_of_purchase': ['nunique', 'mean', 'min', 'max'],
    'Minute_of_purchase': ['nunique', 'mean', 'min', 'max'],
    'Second_of_purchase': ['nunique', 'mean', 'min', 'max'],
    'weekofyear': ['nunique', 'mean', 'min', 'max'],
    'month_diff': ['max','min','mean','var','skew'],
    'day': ['nunique', 'sum', 'min'],
    'month' : ['sum', 'mean', 'nunique','max'],
    'purchase_amount_quantiles' : ['var', 'mean', 'skew'],
    'duration' : ['mean','min','max','var','skew'],
    'amount_month_ratio' : ['mean','min','max','var','skew'],
    'subsector_id': ['nunique'],
    'card_id': ['size'],
    'city_id' : ['nunique'],
    'state_id' : ['nunique'],
    'merchant_id': ['nunique'],
    'installments': ['sum','max','mean','var','skew'],
    'merchant_category_id': ['nunique'],
    'purchase_amount': ['sum', 'mean', 'min', 'max', 'var','skew'],
    'EMI' : ['sum','mean','max','min','var'],
    'category_1' : ['sum','mean', 'max','min'],
    'category_2' : ['sum','mean'],
    'category_3' : ['sum','mean'],
    'month_lag' : ['sum','max','min','mean','var','skew']
}

aggregated_trans_1 = new_hist_trans.groupby('card_id').agg(aggregations)
aggregated_trans_1.columns = ['new_transactions_'+'_'.join(col).strip()
                           for col in aggregated_trans_1.columns.values]
aggregated_trans_1.reset_index(inplace=True)

# extracting some more features based on aggregated features.
aggregated_trans_1['new_transactions_purchase_date_diff'] = (aggregated_trans_1['new_transactions_purchase_date_max']-aggregated_trans_1['new_transactions_purchase_date_min']).dt.days
aggregated_trans_1['new_transactions_purchase_date_average'] = aggregated_trans_1['new_transactions_purchase_date_diff']/aggregated_trans_1['new_transactions_card_id_size']
aggregated_trans_1['new_transactions_purchase_date_uptonow'] = (datetime.datetime.today()-aggregated_trans_1['new_transactions_purchase_date_max']).dt.days
aggregated_trans_1['new_transactions_purchase_date_uptomin'] = (datetime.datetime.today()-aggregated_trans_1['new_transactions_purchase_date_min']).dt.days

Memory usage after optimization is: 114.20 MB
Decreased by 45.5%
Memory usage after optimization is: 110.45 MB
Decreased by 69.4%


In [ ]:
aggregated_trans_1.head()

,card_id,new_transactions_purchase_date_max,new_transactions_purchase_date_min,new_transactions_purchased_on_weekend_sum,new_transactions_purchased_on_weekend_mean,new_transactions_purchased_on_weekday_sum,new_transactions_purchased_on_weekday_mean,new_transactions_dayofweek_nunique,new_transactions_dayofweek_sum,new_transactions_dayofweek_mean,new_transactions_dayofweek_max,new_transactions_hour_of_purchase_nunique,new_transactions_hour_of_purchase_mean,new_transactions_hour_of_purchase_min,new_transactions_hour_of_purchase_max,new_transactions_Minute_of_purchase_nunique,new_transactions_Minute_of_purchase_mean,new_transactions_Minute_of_purchase_min,new_transactions_Minute_of_purchase_max,new_transactions_Second_of_purchase_nunique,new_transactions_Second_of_purchase_mean,new_transactions_Second_of_purchase_min,new_transactions_Second_of_purchase_max,new_transactions_weekofyear_nunique,new_transactions_weekofyear_mean,new_transactions_weekofyear_min,new_transactions_weekofyear_max,new_transactions_month_diff_max,new_transactions_month_diff_min,new_transactions_month_diff_mean,new_transactions_month_diff_var,new_transactions_month_diff_skew,new_transactions_day_nunique,new_transactions_day_sum,new_transactions_day_min,new_transactions_month_sum,new_transactions_month_mean,new_transactions_month_nunique,new_transactions_month_max,new_transactions_purchase_amount_quantiles_var,...,new_transactions_subsector_id_nunique,new_transactions_card_id_size,new_transactions_city_id_nunique,new_transactions_state_id_nunique,new_transactions_merchant_id_nunique,new_transactions_installments_sum,new_transactions_installments_max,new_transactions_installments_mean,new_transactions_installments_var,new_transactions_installments_skew,new_transactions_merchant_category_id_nunique,new_transactions_purchase_amount_sum,new_transactions_purchase_amount_mean,new_transactions_purchase_amount_min,new_transactions_purchase_amount_max,new_transactions_purchase_amount_var,new_transactions_purchase_amount_skew,new_transactions_EMI_sum,new_transactions_EMI_mean,new_transactions_EMI_max,new_transactions_EMI_min,new_transactions_EMI_var,new_transactions_category_1_sum,new_transactions_category_1_mean,new_transactions_category_1_max,new_transactions_category_1_min,new_transactions_category_2_sum,new_transactions_category_2_mean,new_transactions_category_3_sum,new_transactions_category_3_mean,new_transactions_month_lag_sum,new_transactions_month_lag_max,new_transactions_month_lag_min,new_transactions_month_lag_mean,new_transactions_month_lag_var,new_transactions_month_lag_skew,new_transactions_purchase_date_diff,new_transactions_purchase_date_average,new_transactions_purchase_date_uptonow,new_transactions_purchase_date_uptomin
0,C_ID_00007093c1,2018-04-09 16:23:59,2018-04-03 11:13:35,0,0.000000,2,1.000000,2,1.0,0.500000,1,2,13.500000,11,16,2,18.000000,13,23,2,47.000000,35,59,2,14.500000,14,15,35,35,35.000000,0.000000,NaN,2,12.0,3,8.0,4.000000,1,4,0.000000,...,2,2,2,2,2,2.000000,1.000000,1.000000,0.000000,NaN,2,-1.328125,-0.664062,-0.671875,-0.656738,0.000115,NaN,-1.328125,-0.664062,-0.656738,-0.671875,0.000115,0,0.000000,0,0,4.0,2.000000,2,1.000000,4.0,2,2,2.000000,0.000000,NaN,6,3.000000,1011,1018
1,C_ID_0001238066,2018-04-30 19:57:30,2018-03-01 16:48:27,12,0.461538,14,0.538462,6,105.0,4.038462,6,16,15.192308,0,23,20,29.884615,2,57,24,30.615385,0,57,9,12.846154,9,18,36,35,35.076923,0.073846,3.373242,14,474.0,1,87.0,3.346154,2,4,1.618462,...,9,26,8,4,25,56.000000,13.000000,2.154297,8.937500,2.896484,15,-14.851562,-0.571289,-0.740723,-0.078308,0.030075,1.569336,-13.445312,-0.517090,-0.011688,-0.740723,0.060730,2,0.076923,1,0,38.0,1.461914,29,1.115385,35.0,2,1,1.346154,0.235385,0.687052,60,2.307692,990,1050
2,C_ID_0001506ef0,2018-03-22 09:14:30,2018-03-16 22:21:58,0,0.000000,2,1.000000,2,7.0,3.500000,4,2,15.500000,9,22,2,17.500000,14,21,2,44.000000,30,58,2,11.500000,11,12,35,35,35.000000,0.000000,NaN,2,38.0,16,6.0,3.000000,1,3,0.500000,...,2,2,1,1,2,0.199951,0.0

In [ ]:
aggregated_trans_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290001 entries, 0 to 290000
Data columns (total 92 columns):
 #   Column                                           Non-Null Count   Dtype         
---  ------                                           --------------   -----         
 0   card_id                                          290001 non-null  object        
 1   new_transactions_purchase_date_max               290001 non-null  datetime64[ns]
 2   new_transactions_purchase_date_min               290001 non-null  datetime64[ns]
 3   new_transactions_purchased_on_weekend_sum        290001 non-null  int8          
 4   new_transactions_purchased_on_weekend_mean       290001 non-null  float64       
 5   new_transactions_purchased_on_weekday_sum        290001 non-null  int8          
 6   new_transactions_purchased_on_weekday_mean       290001 non-null  float64       
 7   new_transactions_dayofweek_nunique               290001 non-null  int64         
 8   new_transactions_dayofwe

## Merging transactions with train and test

In [ ]:
train1=pd.merge(train, aggregated_trans, on='card_id', how='left')
test1=pd.merge(test, aggregated_trans, on='card_id', how='left')

In [ ]:
train1.shape,test1.shape

((201917, 116), (123623, 114))

In [ ]:
train1=pd.merge(train1, aggregated_trans_1, on='card_id', how='left')
test1=pd.merge(test1, aggregated_trans_1, on='card_id', how='left')

In [ ]:
print(train1.shape)
print(test1.shape)

(201917, 207)
(123623, 205)


In [ ]:
train1.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,quarter,total_time,start_month,start_year,dayofweek,total_time_feature1,total_time_feature2,total_time_feature3,total_time_feature1_ratio,total_time_feature2_ratio,total_time_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var,transactions_purchase_date_max,transactions_purchase_date_min,transactions_purchased_on_weekend_sum,transactions_purchased_on_weekend_mean,transactions_purchased_on_weekday_sum,transactions_purchased_on_weekday_mean,transactions_dayofweek_nunique,transactions_dayofweek_sum,transactions_dayofweek_mean,transactions_dayofweek_max,transactions_hour_of_purchase_nunique,transactions_hour_of_purchase_mean,transactions_hour_of_purchase_min,transactions_hour_of_purchase_max,transactions_Minute_of_purchase_nunique,transactions_Minute_of_purchase_mean,transactions_Minute_of_purchase_min,...,new_transactions_subsector_id_nunique,new_transactions_card_id_size,new_transactions_city_id_nunique,new_transactions_state_id_nunique,new_transactions_merchant_id_nunique,new_transactions_installments_sum,new_transactions_installments_max,new_transactions_installments_mean,new_transactions_installments_var,new_transactions_installments_skew,new_transactions_merchant_category_id_nunique,new_transactions_purchase_amount_sum,new_transactions_purchase_amount_mean,new_transactions_purchase_amount_min,new_transactions_purchase_amount_max,new_transactions_purchase_amount_var,new_transactions_purchase_amount_skew,new_transactions_EMI_sum,new_transactions_EMI_mean,new_transactions_EMI_max,new_transactions_EMI_min,new_transactions_EMI_var,new_transactions_category_1_sum,new_transactions_category_1_mean,new_transactions_category_1_max,new_transactions_category_1_min,new_transactions_category_2_sum,new_transactions_category_2_mean,new_transactions_category_3_sum,new_transactions_category_3_mean,new_transactions_month_lag_sum,new_transactions_month_lag_max,new_transactions_month_lag_min,new_transactions_month_lag_mean,new_transactions_month_lag_var,new_transactions_month_lag_skew,new_transactions_purchase_date_diff,new_transactions_purchase_date_average,new_transactions_purchase_date_uptonow,new_transactions_purchase_date_uptomin
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820312,0,2,1324,6,2017,3,17.403471,11.587808,15.130037,0.000010,0.000007,0.000009,0.033324,0.011108,0.013145,0.008752,0.002214,2018-02-25 09:31:15,2017-06-27 14:18:08,90.0,0.346154,170.0,0.653846,7,835.0,3.211538,6,23,13.315385,0,23,60,28.165385,0,...,10.0,23.0,3.0,1.0,23.0,2.298828,0.099976,0.099976,0.000000,0.000000,14.0,-13.242188,-0.575684,-0.724609,-0.296143,0.018433,0.895996,-132.375000,-5.757812,-2.960938,-7.246094,1.843750,0.0,0.000000,0.0,0.0,23.0,1.000000,0.0,0.000000,34.0,2.0,1.0,1.478261,0.260870,0.093233,54.0,2.347826,992.0,1047.0
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392822,0,1,1475,1,2017,6,15.799598,16.792542,15.167182,0.000007,0.000008,0.000007,0.032379,0.010793,0.011385,0.010283,0.000555,2018-01-31 22:31:09,2017-01-06 16:29:42,132.0,0.377143,218.0,0.622857,7,1177.0,3.362857,6,24,14.717143,0,23,60,29.134286,0,...,4.0,6.0,1.0,1.0,6.0,6.000000,1.000000,1.000000,0.000000,0.000000,5.0,-4.355469,-0.726074,-0.739258,-0.701660,0.000207,1.255859,-4.355469,-0.726074,-0.701660,-0.739258,0.000207,0.0,0.000000,0.0,0.0,6.0,1.000000,6.0,1.000000,9.0,2.0,1.0,1.500000,0.300000,0.000000,56.0,9.333333,1022.0,1078.0
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.687988,0,3,1628,8,2016,0,17.272900,14.248453,16.740455,0.000007,0.000005,0.000006,0.029645,0.009882,0.010610,0.008752,0.000992,2018-02-27 19:08:25,2017-01-11 08:21:22,11.0,0.255814,32.0,0.744186,7,142.0,3.302326,6,14,17.906977,8,23,33,25.674419,0,...,1.0,1.0,1.0,1.0,1.0,0.099976,0.099976,0.099976,NaN,NaN,1.0,-0.700195,-0.700195,-0.700195,-0.700195,NaN,NaN,-7.000000,-7.000000,-7.000000,-7.000000,NaN,0.0,0.000000,0.0,0.0,5.0,5.000000,0.0,0.000000,2.0,2.0,2.0,2.000000,NaN,NaN,0

In [ ]:
test1.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter,total_time,start_month,start_year,dayofweek,total_time_feature1,total_time_feature2,total_time_feature3,total_time_feature1_ratio,total_time_feature2_ratio,total_time_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var,transactions_purchase_date_max,transactions_purchase_date_min,transactions_purchased_on_weekend_sum,transactions_purchased_on_weekend_mean,transactions_purchased_on_weekday_sum,transactions_purchased_on_weekday_mean,transactions_dayofweek_nunique,transactions_dayofweek_sum,transactions_dayofweek_mean,transactions_dayofweek_max,transactions_hour_of_purchase_nunique,transactions_hour_of_purchase_mean,transactions_hour_of_purchase_min,transactions_hour_of_purchase_max,transactions_Minute_of_purchase_nunique,transactions_Minute_of_purchase_mean,transactions_Minute_of_purchase_min,transactions_Minute_of_purchase_max,transactions_Second_of_purchase_nunique,...,new_transactions_subsector_id_nunique,new_transactions_card_id_size,new_transactions_city_id_nunique,new_transactions_state_id_nunique,new_transactions_merchant_id_nunique,new_transactions_installments_sum,new_transactions_installments_max,new_transactions_installments_mean,new_transactions_installments_var,new_transactions_installments_skew,new_transactions_merchant_category_id_nunique,new_transactions_purchase_amount_sum,new_transactions_purchase_amount_mean,new_transactions_purchase_amount_min,new_transactions_purchase_amount_max,new_transactions_purchase_amount_var,new_transactions_purchase_amount_skew,new_transactions_EMI_sum,new_transactions_EMI_mean,new_transactions_EMI_max,new_transactions_EMI_min,new_transactions_EMI_var,new_transactions_category_1_sum,new_transactions_category_1_mean,new_transactions_category_1_max,new_transactions_category_1_min,new_transactions_category_2_sum,new_transactions_category_2_mean,new_transactions_category_3_sum,new_transactions_category_3_mean,new_transactions_month_lag_sum,new_transactions_month_lag_max,new_transactions_month_lag_min,new_transactions_month_lag_mean,new_transactions_month_lag_var,new_transactions_month_lag_skew,new_transactions_purchase_date_diff,new_transactions_purchase_date_average,new_transactions_purchase_date_uptonow,new_transactions_purchase_date_uptomin
0,2017-04-01,C_ID_0ab67a22ab,0.010479,0.014166,0.011428,2,1385,4,2017,5,14.513952,19.620467,15.827116,0.000008,0.000010,0.000008,0.036073,0.012024,0.014166,0.010479,0.001915,2017-12-29 01:05:24,2017-04-04 18:22:24,12.0,0.176471,56.0,0.823529,7,190.0,2.794118,6,17,14.367647,0,23,38,30.573529,0,55,43,...,3.0,3.0,3.0,1.0,3.0,5.0,3.0,1.666992,1.333008,1.729492,3.0,-1.777344,-0.592285,-0.722168,-0.383301,0.033447,1.577148,-1.521484,-0.507324,-0.127808,-0.722168,0.108643,0.0,0.000000,0.0,0.0,3.0,1.000000,4.0,1.333333,6.0,2.0,2.0,2.000000,0.000000,0.000000,25.0,8.333333,1051.0,1077.0
1,2017-01-01,C_ID_130fd0cbdd,0.010610,0.014166,0.010283,1,1475,1,2017,6,15.649587,20.895443,15.167182,0.000007,0.000010,0.000007,0.035059,0.011686,0.014166,0.010283,0.002154,2018-02-18 21:30:11,2017-01-13 15:31:38,17.0,0.217949,61.0,0.782051,7,229.0,2.935897,6,18,14.423077,0,23,45,31.205128,0,59,40,...,6.0,9.0,2.0,2.0,9.0,11.0,3.0,1.222656,0.444336,2.998047,8.0,-5.945312,-0.660645,-0.740723,-0.506348,0.005062,1.287109,-5.605469,-0.623047,-0.168823,-0.740723,0.030731,2.0,0.222222,1.0,0.0,30.0,3.333984,10.0,1.111111,13.0,2.0,1.0,1.444444,0.277778,0.271052,48.0,5.333333,1000.0,1049.0
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011385,0.011428,3,1263,8,2017,1,16.601649,14.378970,14.432959,0.000010,0.000009,0.000009,0.035957,0.011986,0.013145,0.011385,0.001004,2018-02-02 18:57:03,2017-08-25 14:55:37,0.0,0.000000,13.0,1.000000,4,34.0,2.615385,4,7,17.000000,13,20,13,31.769231,2,57,13,...,2.0,2.0,2.0,2.0,2.0,11.0,10.0,5.500000,40.500000,NaN,2.0,0.179688,0.089844,-0.724609,0.904297,1.327148,NaN,-0.634277,-0.317139,0.090454,-0.724609,0.332275,1.0,0.500000,1.0,0.0,6.0,3.000000,3.0,1.500000,2.0,1.0,1.0,

In [ ]:
# converting engineered date features to datetime so that we can use them afterwards.
train1['transactions_purchase_date_max'] = pd.to_datetime(train1['transactions_purchase_date_max'])
train1['transactions_purchase_date_min'] = pd.to_datetime(train1['transactions_purchase_date_min'])
train1['new_transactions_purchase_date_max'] = pd.to_datetime(train1['new_transactions_purchase_date_max'])
train1['new_transactions_purchase_date_min'] = pd.to_datetime(train1['new_transactions_purchase_date_min'])

test1['transactions_purchase_date_max'] = pd.to_datetime(test1['transactions_purchase_date_max'])
test1['transactions_purchase_date_min'] = pd.to_datetime(test1['transactions_purchase_date_min'])
test1['new_transactions_purchase_date_max'] = pd.to_datetime(test1['new_transactions_purchase_date_max'])
test1['new_transactions_purchase_date_min'] = pd.to_datetime(test1['new_transactions_purchase_date_min'])

Extracting some more features from train

In [ ]:
# extracting some more features from train by performing some simple caluculations.
# inspired by https://www.kaggle.com/mfjwr1/simple-lightgbm-without-blending
train1['transactions_purchase_date_difference']=train1['transactions_purchase_date_max'] - train1['transactions_purchase_date_min']
train1['new_transactions_purchase_date_difference'] = train1['new_transactions_purchase_date_max'] - train1['new_transactions_purchase_date_min']
train1['Avg_purchase'] = train1['transactions_purchase_date_difference'] / train1['transactions_card_id_size']
train1['new_Avg_purchase'] = train1['new_transactions_purchase_date_difference'] / train1['new_transactions_card_id_size']
train1['last_purchase_from_now'] = (datetime.datetime.today() - train1['transactions_purchase_date_max']).dt.days
train1['new_last_purchase_from_now'] = (datetime.datetime.today() - train1['new_transactions_purchase_date_max']).dt.days
train1['first_purchase_from_now'] = (datetime.datetime.today() - train1['transactions_purchase_date_min']).dt.days
train1['new_first_purchase_from_now'] = (datetime.datetime.today() - train1['new_transactions_purchase_date_min']).dt.days

In [ ]:
# extracting some more features from train by performing some simple caluculations.
# inspired by https://www.kaggle.com/mfjwr1/simple-lightgbm-without-blending
train1['card_id_total'] = train1['new_transactions_card_id_size']+train1['transactions_card_id_size']
train1['card_id_ratio'] =  train1['new_transactions_card_id_size']/train1['transactions_card_id_size']

train1['total_purchase_amount_max'] = train1['new_transactions_purchase_amount_max']+train1['transactions_purchase_amount_max']
train1['total_purchase_amount_min'] = train1['new_transactions_purchase_amount_min']+train1['transactions_purchase_amount_min']
train1['total_purchase_amount_mean'] = train1['new_transactions_purchase_amount_mean']+train1['transactions_purchase_amount_mean']
train1['total_purchase_amount_sum'] = train1['new_transactions_purchase_amount_sum']+train1['transactions_purchase_amount_sum']
train1['total_purchase_amount_ratio'] = train1['new_transactions_purchase_amount_sum']/train1['transactions_purchase_amount_sum']

train1['total_installments_max'] = train1['new_transactions_installments_max'] + train1['transactions_installments_max']
train1['total_installments_mean'] = train1['new_transactions_installments_mean'] + train1['transactions_installments_mean']
train1['total_installments_sum'] = train1['new_transactions_installments_sum'] + train1['transactions_installments_sum']
train1['total_installments_ratio'] = train1['new_transactions_installments_sum'] / train1['transactions_installments_sum']

train1['total_month_lag_max'] = train1['new_transactions_month_lag_max'] + train1['transactions_month_lag_max']
train1['total_month_lag_min'] = train1['new_transactions_month_lag_min'] + train1['transactions_month_lag_min']
train1['total_month_lag_mean'] = train1['new_transactions_month_lag_mean'] + train1['transactions_month_lag_mean']
train1['total_month_lag_sum'] = train1['new_transactions_month_lag_sum'] + train1['transactions_month_lag_sum']
train1['total_month_lag_ratio'] = train1['new_transactions_month_lag_sum'] / train1['transactions_month_lag_sum']

train1['total_duration_max'] = train1['new_transactions_duration_max'] + train1['transactions_duration_max']
train1['total_duration_min'] = train1['new_transactions_duration_min'] + train1['transactions_duration_min']
train1['total_duration_mean'] = train1['new_transactions_duration_mean'] + train1['transactions_duration_mean']


train1['total_month_diff_max'] = train1['new_transactions_month_diff_max'] + train1['transactions_month_diff_max']
train1['total_month_diff_mean'] = train1['new_transactions_month_diff_mean'] + train1['transactions_month_diff_mean']
train1['total_month_diff_min'] = train1['new_transactions_month_diff_min'] + train1['transactions_month_diff_min']

train1['total_amount_month_ratio_max'] = train1['new_transactions_amount_month_ratio_max'] + train1['transactions_amount_month_ratio_max']
train1['total_amount_month_ratio_min'] = train1['new_transactions_amount_month_ratio_min'] + train1['transactions_amount_month_ratio_min']
train1['total_amount_month_ratio_mean'] = train1['new_transactions_amount_month_ratio_mean'] + train1['transactions_amount_month_ratio_mean']

train1['customer_rating'] = train1['transactions_card_id_size'] * train1['transactions_purchase_amount_sum'] / train1['transactions_month_diff_mean']
train1['new_customer_rating'] = train1['new_transactions_card_id_size'] * train1['new_transactions_purchase_amount_sum'] / train1['new_transactions_month_diff_mean']
train1['customer_rating_ratio'] = train1['customer_rating'] / train1['new_customer_rating']

In [ ]:
train1.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,quarter,total_time,start_month,start_year,dayofweek,total_time_feature1,total_time_feature2,total_time_feature3,total_time_feature1_ratio,total_time_feature2_ratio,total_time_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var,transactions_purchase_date_max,transactions_purchase_date_min,transactions_purchased_on_weekend_sum,transactions_purchased_on_weekend_mean,transactions_purchased_on_weekday_sum,transactions_purchased_on_weekday_mean,transactions_dayofweek_nunique,transactions_dayofweek_sum,transactions_dayofweek_mean,transactions_dayofweek_max,transactions_hour_of_purchase_nunique,transactions_hour_of_purchase_mean,transactions_hour_of_purchase_min,transactions_hour_of_purchase_max,transactions_Minute_of_purchase_nunique,transactions_Minute_of_purchase_mean,transactions_Minute_of_purchase_min,...,new_transactions_purchase_date_diff,new_transactions_purchase_date_average,new_transactions_purchase_date_uptonow,new_transactions_purchase_date_uptomin,transactions_purchase_date_difference,new_transactions_purchase_date_difference,Avg_purchase,new_Avg_purchase,last_purchase_from_now,new_last_purchase_from_now,first_purchase_from_now,new_first_purchase_from_now,card_id_total,card_id_ratio,total_purchase_amount_max,total_purchase_amount_min,total_purchase_amount_mean,total_purchase_amount_sum,total_purchase_amount_ratio,total_installments_max,total_installments_mean,total_installments_sum,total_installments_ratio,total_month_lag_max,total_month_lag_min,total_month_lag_mean,total_month_lag_sum,total_month_lag_ratio,total_duration_max,total_duration_min,total_duration_mean,total_month_diff_max,total_month_diff_mean,total_month_diff_min,total_amount_month_ratio_max,total_amount_month_ratio_min,total_amount_month_ratio_mean,customer_rating,new_customer_rating,customer_rating_ratio
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820312,0,2,1324,6,2017,3,17.403471,11.587808,15.130037,0.000010,0.000007,0.000009,0.033324,0.011108,0.013145,0.008752,0.002214,2018-02-25 09:31:15,2017-06-27 14:18:08,90.0,0.346154,170.0,0.653846,7,835.0,3.211538,6,23,13.315385,0,23,60,28.165385,0,...,54.0,2.347826,992.0,1047.0,242 days 19:13:07,54 days 21:18:29,0 days 22:24:44.565384615,2 days 09:16:27.347826086,1055,992.0,1298,1047.0,283.0,0.088462,0.923828,-1.463867,-1.218750,-180.2500,0.079285,1.099609,0.213867,31.890625,0.077698,2.0,-7.0,-2.433278,-983.0,-0.033432,32.312500,-51.78125,-42.68750,71.0,70.080769,70.0,0.026398,-0.041809,-0.034760,-1237.715163,-8.702009,142.233267
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392822,0,1,1475,1,2017,6,15.799598,16.792542,15.167182,0.000007,0.000008,0.000007,0.032379,0.010793,0.011385,0.010283,0.000555,2018-01-31 22:31:09,2017-01-06 16:29:42,132.0,0.377143,218.0,0.622857,7,1177.0,3.362857,6,24,14.717143,0,23,60,29.134286,0,...,56.0,9.333333,1022.0,1078.0,390 days 06:01:27,56 days 13:40:32,1 days 02:45:36.248571428,9 days 10:16:45.333333333,1079,1022.0,1469,1078.0,356.0,0.017143,0.518066,-1.481445,-1.337891,-218.5000,0.020340,14.000000,2.632812,577.000000,0.010506,2.0,-11.0,-3.531429,-1752.0,-0.005111,18.640625,-53.90625,-48.21875,73.0,72.105714,71.0,0.014389,-0.041504,-0.037109,-2075.675595,-0.725911,2859.406022
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.687988,0,3,1628,8,2016,0,17.272900,14.248453,16.740455,0.000007,0.000005,0.000006,0.029645,0.009882,0.010610,0.008752,0.000992,2018-02-27 19:08:25,2017-01-11 08:21:22,11.0,0.255814,32.0,0.744186,7,142.0,3.302326,6,14,17.906977,8,23,33,25.674419,0,...,0.0,0.000000,992.0,992.0,412 days 10:47:03,0 days 00:00:00,9 days 14:12:15.418604651,0 days 00:00:00,1052,992.0,1465,992.0,44.0,0.023256,-0.846191,-1.429688,-1.378906,-29.8750,0.024002,0.199951,0.199951,4.402344,0.023239,2.0,-11.0,-6.604651,-368.0,-0.005405,-29.609375,-50.68750,-48.28125,71.0,70.069767,70.0,-0.024170,-0.040863,-0.039337,-35.768433,-0.020006,1787.922785
3,2017-09-01,

Extracting some more features from test

In [ ]:
# converting engineered date features to datetime so that we can use them afterwards.
test1['transactions_purchase_date_difference']=test1['transactions_purchase_date_max'] - test1['transactions_purchase_date_min']
test1['new_transactions_purchase_date_difference'] = test1['new_transactions_purchase_date_max'] - test1['new_transactions_purchase_date_min']
test1['Avg_purchase'] = test1['transactions_purchase_date_difference'] / test1['transactions_card_id_size']
test1['new_Avg_purchase'] = test1['new_transactions_purchase_date_difference'] / test1['new_transactions_card_id_size']
test1['last_purchase_from_now'] = (datetime.datetime.today() - test1['transactions_purchase_date_max']).dt.days
test1['new_last_purchase_from_now'] = (datetime.datetime.today() - test1['new_transactions_purchase_date_max']).dt.days
test1['first_purchase_from_now'] = (datetime.datetime.today() - test1['transactions_purchase_date_min']).dt.days
test1['new_first_purchase_from_now'] = (datetime.datetime.today() - test1['new_transactions_purchase_date_min']).dt.days

In [ ]:
# extracting some more features from train by performing some simple caluculations.
# inspired by https://www.kaggle.com/mfjwr1/simple-lightgbm-without-blending
test1['card_id_total'] = test1['new_transactions_card_id_size']+test1['transactions_card_id_size']
test1['card_id_ratio'] =  test1['new_transactions_card_id_size']/test1['transactions_card_id_size']

test1['total_purchase_amount_max'] = test1['new_transactions_purchase_amount_max']+test1['transactions_purchase_amount_max']
test1['total_purchase_amount_min'] = test1['new_transactions_purchase_amount_min']+test1['transactions_purchase_amount_min']
test1['total_purchase_amount_mean'] = test1['new_transactions_purchase_amount_mean']+test1['transactions_purchase_amount_mean']
test1['total_purchase_amount_sum'] = test1['new_transactions_purchase_amount_sum']+test1['transactions_purchase_amount_sum']
test1['total_purchase_amount_ratio'] = test1['new_transactions_purchase_amount_sum']/test1['transactions_purchase_amount_sum']

test1['total_installments_max'] = test1['new_transactions_installments_max'] + test1['transactions_installments_max']
test1['total_installments_mean'] = test1['new_transactions_installments_mean'] + test1['transactions_installments_mean']
test1['total_installments_sum'] = test1['new_transactions_installments_sum'] + test1['transactions_installments_sum']
test1['total_installments_ratio'] = test1['new_transactions_installments_sum'] / test1['transactions_installments_sum']

test1['total_month_lag_max'] = test1['new_transactions_month_lag_max'] + test1['transactions_month_lag_max']
test1['total_month_lag_min'] = test1['new_transactions_month_lag_min'] + test1['transactions_month_lag_min']
test1['total_month_lag_mean'] = test1['new_transactions_month_lag_mean'] + test1['transactions_month_lag_mean']
test1['total_month_lag_sum'] = test1['new_transactions_month_lag_sum'] + test1['transactions_month_lag_sum']
test1['total_month_lag_ratio'] = test1['new_transactions_month_lag_sum'] / test1['transactions_month_lag_sum']

test1['total_duration_max'] = test1['new_transactions_duration_max'] + test1['transactions_duration_max']
test1['total_duration_min'] = test1['new_transactions_duration_min'] + test1['transactions_duration_min']
test1['total_duration_mean'] = test1['new_transactions_duration_mean'] + test1['transactions_duration_mean']

test1['total_month_diff_max'] = test1['new_transactions_month_diff_max'] + test1['transactions_month_diff_max']
test1['total_month_diff_mean'] = test1['new_transactions_month_diff_mean'] + test1['transactions_month_diff_mean']
test1['total_month_diff_min'] = test1['new_transactions_month_diff_min'] + test1['transactions_month_diff_min']

test1['total_amount_month_ratio_max'] = test1['new_transactions_amount_month_ratio_max'] + test1['transactions_amount_month_ratio_max']
test1['total_amount_month_ratio_min'] = test1['new_transactions_amount_month_ratio_min'] + test1['transactions_amount_month_ratio_min']
test1['total_amount_month_ratio_mean'] = test1['new_transactions_amount_month_ratio_mean'] + test1['transactions_amount_month_ratio_mean']

test1['customer_rating'] = test1['transactions_card_id_size'] * test1['transactions_purchase_amount_sum'] / test1['transactions_month_diff_mean']
test1['new_customer_rating'] = test1['new_transactions_card_id_size'] * test1['new_transactions_purchase_amount_sum'] / test1['new_transactions_month_diff_mean']
test1['customer_rating_ratio'] = test1['customer_rating'] / test1['new_customer_rating']

In [ ]:
test1.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter,total_time,start_month,start_year,dayofweek,total_time_feature1,total_time_feature2,total_time_feature3,total_time_feature1_ratio,total_time_feature2_ratio,total_time_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var,transactions_purchase_date_max,transactions_purchase_date_min,transactions_purchased_on_weekend_sum,transactions_purchased_on_weekend_mean,transactions_purchased_on_weekday_sum,transactions_purchased_on_weekday_mean,transactions_dayofweek_nunique,transactions_dayofweek_sum,transactions_dayofweek_mean,transactions_dayofweek_max,transactions_hour_of_purchase_nunique,transactions_hour_of_purchase_mean,transactions_hour_of_purchase_min,transactions_hour_of_purchase_max,transactions_Minute_of_purchase_nunique,transactions_Minute_of_purchase_mean,transactions_Minute_of_purchase_min,transactions_Minute_of_purchase_max,transactions_Second_of_purchase_nunique,...,new_transactions_purchase_date_diff,new_transactions_purchase_date_average,new_transactions_purchase_date_uptonow,new_transactions_purchase_date_uptomin,transactions_purchase_date_difference,new_transactions_purchase_date_difference,Avg_purchase,new_Avg_purchase,last_purchase_from_now,new_last_purchase_from_now,first_purchase_from_now,new_first_purchase_from_now,card_id_total,card_id_ratio,total_purchase_amount_max,total_purchase_amount_min,total_purchase_amount_mean,total_purchase_amount_sum,total_purchase_amount_ratio,total_installments_max,total_installments_mean,total_installments_sum,total_installments_ratio,total_month_lag_max,total_month_lag_min,total_month_lag_mean,total_month_lag_sum,total_month_lag_ratio,total_duration_max,total_duration_min,total_duration_mean,total_month_diff_max,total_month_diff_mean,total_month_diff_min,total_amount_month_ratio_max,total_amount_month_ratio_min,total_amount_month_ratio_mean,customer_rating,new_customer_rating,customer_rating_ratio
0,2017-04-01,C_ID_0ab67a22ab,0.010479,0.014166,0.011428,2,1385,4,2017,5,14.513952,19.620467,15.827116,0.000008,0.000010,0.000008,0.036073,0.012024,0.014166,0.010479,0.001915,2017-12-29 01:05:24,2017-04-04 18:22:24,12.0,0.176471,56.0,0.823529,7,190.0,2.794118,6,17,14.367647,0,23,38,30.573529,0,55,43,...,25.0,8.333333,1051.0,1077.0,268 days 06:43:00,25 days 09:24:23,3 days 22:41:13.235294117,8 days 11:08:07.666666666,1113,1051.0,1381,1077.0,71.0,0.044118,-0.147583,-1.466797,-1.191406,-42.500000,0.043640,15.0,3.742188,146.0,0.035461,2.0,-6.0,-1.632353,-241.0,-0.024291,-5.226562,-55.00000,-44.12500,75.0,74.102941,74.0,-0.004158,-0.039490,-0.032166,-74.626833,-0.144109,517.850083
1,2017-01-01,C_ID_130fd0cbdd,0.010610,0.014166,0.010283,1,1475,1,2017,6,15.649587,20.895443,15.167182,0.000007,0.000010,0.000007,0.035059,0.011686,0.014166,0.010283,0.002154,2018-02-18 21:30:11,2017-01-13 15:31:38,17.0,0.217949,61.0,0.782051,7,229.0,2.935897,6,18,14.423077,0,23,45,31.205128,0,59,40,...,48.0,5.333333,1000.0,1049.0,401 days 05:58:33,48 days 05:41:29,5 days 03:27:40.423076923,5 days 08:37:56.555555555,1061,1000.0,1462,1049.0,87.0,0.115385,-0.187500,-1.472656,-1.291016,-55.062500,0.121033,7.0,2.287109,94.0,0.132568,2.0,-12.0,-8.965812,-799.0,-0.016010,-6.562500,-52.25000,-45.25000,71.0,70.153846,70.0,-0.005356,-0.042053,-0.036804,-108.999453,-1.528795,71.297642
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011385,0.011428,3,1263,8,2017,1,16.601649,14.378970,14.432959,0.000010,0.000009,0.000009,0.035957,0.011986,0.013145,0.011385,0.001004,2018-02-02 18:57:03,2017-08-25 14:55:37,0.0,0.000000,13.0,1.000000,4,34.0,2.615385,4,7,17.000000,13,20,13,31.769231,2,57,13,...,11.0,5.500000,1039.0,1050.0,161 days 04:01:26,11 days 22:19:13,12 days 09:32:25.076923076,5 days 23:09:36.500000,1077,1039.0,1238,1050.0,15.0,0.153846,2.125000,-1.261719,0.315674,3.115234,0.061218,23.0,12.117188,97.0,0.127930,1.0,-5.0,-1.076923,-25.0,-0.074074,75.250000,-44.12500,11.50000,71.0,70.500000,70.0,0.059967,-0.036041,0.008652,1.090346,0.010123,107.707220


In [ ]:
# replacing inf values with nan. 
train1.replace([-np.inf,np.inf], np.nan, inplace=True)
test1.replace([-np.inf,np.inf], np.nan, inplace=True)

In [ ]:
# checking for nan values.
k= train1.columns[train1.isna().any()]
v= test1.columns[test1.isna().any()]

In [ ]:
len(k),len(v)

(129, 129)

In [ ]:
for i in range(len(k)):
  train1[k[i]].fillna(train1[k[i]].mode()[0], inplace=True)
  test1[k[i]].fillna(test1[k[i]].mode()[0], inplace=True)

In [ ]:
train1.columns[train1.isnull().any()]

Index([], dtype='object')

In [ ]:
test1.columns[test1.isna().any()]

Index([], dtype='object')

In [ ]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201917 entries, 0 to 201916
Columns: 243 entries, first_active_month to customer_rating_ratio
dtypes: datetime64[ns](5), float16(64), float64(128), int64(24), int8(17), object(1), timedelta64[ns](4)
memory usage: 279.0+ MB


In [ ]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123623 entries, 0 to 123622
Columns: 241 entries, first_active_month to customer_rating_ratio
dtypes: datetime64[ns](5), float16(63), float64(128), int64(23), int8(17), object(1), timedelta64[ns](4)
memory usage: 169.7+ MB


We can see that there are datetime64[ns] and timedelta64[ns] present in dataset which is undesirable for modelling.


In [ ]:
# getting columns having datetime64[ns] datatypes
types=train1.select_dtypes(include=['datetime64[ns]']).columns

In [ ]:
# removing columns having datetime64[ns] datatypes
train1=train1.drop(types,axis=1)
test1=test1.drop(types,axis=1)

In [ ]:
# # getting columns having timedelta64[ns] datatypes
types1=train1.select_dtypes(include=['timedelta64[ns]']).columns

In [ ]:
# changing timedelta64[ns] to int64 datatype so that we can perform modelling
for i in types1:
 train1[i] = train1[i].astype(np.int64) * 1e-9
for i in types1:
 test1[i] = test1[i].astype(np.int64) * 1e-9

In [ ]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201917 entries, 0 to 201916
Columns: 238 entries, card_id to customer_rating_ratio
dtypes: float16(64), float64(132), int64(24), int8(17), object(1)
memory usage: 271.3+ MB


In [ ]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123623 entries, 0 to 123622
Columns: 236 entries, card_id to customer_rating_ratio
dtypes: float16(63), float64(132), int64(23), int8(17), object(1)
memory usage: 164.9+ MB


Now we can seee that all the dataypes are desirable for modelling.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train1.shape,test1.shape

((201917, 238), (123623, 236))

In [ ]:
# saving the files to drive
train1.to_csv('/content/drive/MyDrive/Colab Notebooks/CASE_STUDY_1/train3')
test1.to_csv('/content/drive/MyDrive/Colab Notebooks/CASE_STUDY_1/test3')

we have total of 235 features which we extracted by FE and merging various datsets given to us by elo. Now we are set for modelling.